In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset and preprocess
#pivot_df = pd.read_csv("synthetic_scenario_30_nodes.csv")
#pivot_df = pd.read_csv("simulated_office_environment.csv")
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv")

pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)  # Fill missing values
pivot_df = pivot_df.head(10000)  # Restrict to first 20000 time steps

# Parameters
reward = 0.5
M = 2 # Maximum number of nodes that can be polled
theta = 0.5  # Threshold for reward condition
penalty = -0.5  # Penalty for polling when difference is <= theta
initial_value = 20  # Initial estimate for last polled values

# Set parameters
beta_1 = 0.9  # dEWMA parameter for state value
beta_2 = 0.01 # dEWMA parameter for rate of change

# Extract column names, ensuring "SN" is excluded
columns = [col for col in pivot_df.columns if col != "SN"]
num_nodes = len(columns)  # Number of sensor nodes based on dataset columns
num_time_steps = len(pivot_df)  # Total time steps based on dataset length

# Track the number of times each category is pulled
category_counts = {'Category A': 0, 'Category B': 0}

# Function to calculate Age of Incorrect Information (AoII) at the sink
def calculate_aoii_sink(current_time, last_received_time, last_rate_of_change):
    # Handle potential NaN or Inf values
    time_diff = current_time - last_received_time
    if np.isnan(last_rate_of_change) or np.isinf(last_rate_of_change):
        return 0.0  # Default to zero if rate of change is invalid
    return abs(time_diff * last_rate_of_change)

# Helper function to update state using dEWMA
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    # Handle the case where delta_t is 0 to avoid division by zero
    if delta_t == 0:
        return measured_value, last_rate_of_change  # Return measured value and keep the last rate of change
    
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Helper function to calculate reward
def calculate_reward(measured_value, last_state_value, theta, penalty):
    if abs(measured_value - last_state_value) > theta:
        return reward  # Reward
    return penalty  # Penalty

# Function to extract numeric node ID from column names dynamically, ensuring valid extraction
def extract_node_id(col_name):
    """Extract numeric node ID from column name, handling cases where no digits are found."""
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None  # Return None if no digits are found

# Dynamic Penalty Update algorithm based on Whittle indices
def dynamic_penalty_update(whittle_indices, M, current_lambda):
    # Convert dictionary values to a list of whittle indices, handling any NaN values
    c_values = []
    for v in whittle_indices.values():
        if np.isnan(v):
            c_values.append(-float('inf'))  # Treat NaN as negative infinity (won't be polled)
        else:
            c_values.append(v)
    
    # Identify the set ℰ of nodes where c_i > λ(t)
    eligible_nodes = [i for i, c_i in enumerate(c_values) if c_i > current_lambda]
    
    # If |ℰ| ≤ M, no penalty update needed
    if len(eligible_nodes) <= M:
        return current_lambda
    
    # Sort the c_i values in descending order
    sorted_c_values = sorted(c_values, reverse=True)
    
    # Identify the M-th value
    M_th_value = sorted_c_values[M-1]
    
    # Update penalty to the M-th value
    new_lambda = M_th_value
    
    return new_lambda

# Main function to simulate Whittle AoII with rewards and track transmission counts
def run_simulation_whittle_aoii_dynamic_penalty(pivot_df, columns, M, theta, penalty):
    cumulative_reward = 0  # Track total cumulative reward
    cumulative_rewards = []  # Store cumulative average reward over time
    last_update_times = {col: 0 for col in columns}  # Last update time for each node
    state_node = {col: np.array([20.0, 0.1]) for col in columns}  # Node states
    
    # Initialize dynamic penalty (λ) to 0
    aoii_penalty = 0.0
    
    # Track penalty evolution
    penalty_values = [aoii_penalty]
    
    # Track nodes polled at each time step
    nodes_polled_count = []
    
    # Set minimum timestamp difference to avoid division by zero
    min_delta_t = 1  # Minimum time difference of 1 to avoid division by zero

    for t in range(len(pivot_df)):
        # Step 1: Compute Whittle indices for each node based on AoII
        whittle_indices = {}
        for col in columns:
            last_state_value, last_rate_of_change = state_node[col]
            measured_value = pivot_df.loc[t, col]

            # Correct AoII calculation at the sink using rate of change
            current_aoii = calculate_aoii_sink(t, last_update_times[col], last_rate_of_change)
            future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], last_rate_of_change)
            future_aoii_active = 0  # AoII resets to 0 if polled

            # Whittle index calculations - with safety checks for NaN/Inf values
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + aoii_penalty
            
            # Calculate the Whittle index with safeguards against invalid values
            whittle_index = q_passive - q_active
            
            # Handle NaN or Inf values
            if np.isnan(whittle_index) or np.isinf(whittle_index):
                whittle_indices[col] = -float('inf')  # Set to negative infinity if invalid
            else:
                whittle_indices[col] = whittle_index

        # Step 2: Update the dynamic penalty (λ) using the algorithm
        aoii_penalty = dynamic_penalty_update(whittle_indices, M, aoii_penalty)
        penalty_values.append(aoii_penalty)
        
        # Step 3: Select nodes to poll based on the updated penalty
        nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= aoii_penalty]
        nodes_polled_count.append(len(nodes_to_poll))

        # Step 4: Poll selected nodes and calculate rewards
        for col in nodes_to_poll:
            measured_value = pivot_df.loc[t, col]
            last_state_value, last_rate_of_change = state_node[col]

            # Calculate reward after polling
            reward = calculate_reward(measured_value, last_state_value, theta, penalty)
            cumulative_reward += reward  # Update cumulative reward

            delta_t_dynamic = max(min_delta_t, t - last_update_times[col])  # Time since last update (ensure at least 1)

            # Update node state and last update time
            state_node[col] = update_node_state_dewma(
                measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1=beta_1, beta_2=beta_2
            )
            last_update_times[col] = t

            # Extract node ID dynamically and categorize
            node_id = extract_node_id(col)
            if node_id is not None:
                if 1 <= node_id <= 5:
                    category_counts['Category A'] += 1
                elif 6 <= node_id <= 10:
                    category_counts['Category B'] += 1
             

        # Step 5: Calculate cumulative average reward
        cumulative_rewards.append(cumulative_reward / (t + 1))

    return cumulative_rewards, category_counts, penalty_values, nodes_polled_count

# Run the simulation with dynamic penalty
cumulative_rewards_whittle, category_pulled_counts, penalty_values, nodes_polled_count = run_simulation_whittle_aoii_dynamic_penalty(
    pivot_df, columns, M, theta, penalty
)

# Print the number of times each category was pulled
print("Transmission Count by Category:")
for category, count in category_pulled_counts.items():
    print(f"{category}: {count} times")

# Save cumulative rewards to CSV
pd.DataFrame(cumulative_rewards_whittle, columns=["cumulative_reward"]).to_csv(
    "cumulative_rewards_whittle_dynamic_penalty.csv", index=False
)



# Calculate statistics about the penalty values and nodes polled
avg_penalty = np.mean(penalty_values)
max_penalty = np.max(penalty_values)
min_penalty = np.min(penalty_values)
avg_nodes_polled = np.mean(nodes_polled_count)
max_nodes_polled = np.max(nodes_polled_count)

print(f"\nDynamic Penalty Statistics:")
print(f"Average Penalty: {avg_penalty:.4f}")
print(f"Maximum Penalty: {max_penalty:.4f}")
print(f"Minimum Penalty: {min_penalty:.4f}")
print(f"\nNodes Polled Statistics:")
print(f"Average Nodes Polled per Time Step: {avg_nodes_polled:.2f}")
print(f"Maximum Nodes Polled at any Time Step: {max_nodes_polled}")
print(f"Target Maximum (M): {M}")

Transmission Count by Category:
Category A: 167 times
Category B: 767 times

Dynamic Penalty Statistics:
Average Penalty: 6.3780
Maximum Penalty: 6.5283
Minimum Penalty: 0.0000

Nodes Polled Statistics:
Average Nodes Polled per Time Step: 0.19
Maximum Nodes Polled at any Time Step: 10
Target Maximum (M): 2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#pivot_df = pd.read_csv("synthetic_scenario_30_nodes.csv")
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv")
pivot_df = pivot_df.head(20000)  # Restrict to first 20000 time steps


M = 2  # Maximum number of nodes that can be polled
aoii_penalty = 0.5
initial_value = 20  # Initial estimate for last polled values

# Set parameters
beta_1 = 0.95  # dEWMA parameter for state value
beta_2 = 0.1 # dEWMA parameter for rate of change

# Extract column names, ensuring "SN" is excluded
columns = [col for col in pivot_df.columns if col != "SN"]
num_nodes = len(columns)  # Number of sensor nodes based on dataset columns
num_time_steps = len(pivot_df)  # Total time steps based on dataset length

# Track the number of times each category is pulled
category_counts = {'Category A': 0, 'Category B': 0}

# Function to calculate Age of Incorrect Information (AoII) at the sink
def calculate_aoii_sink(current_time, last_received_time, last_rate_of_change):
    return abs((current_time - last_received_time) * last_rate_of_change)

# Helper function to update state using dEWMA
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2


# Function to extract numeric node ID from column names dynamically, ensuring valid extraction
def extract_node_id(col_name):
    """Extract numeric node ID from column name, handling cases where no digits are found."""
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None  # Return None if no digits are found

# Main function to simulate Whittle AoII with rewards and track transmission counts
def run_simulation_whittle_aoii(pivot_df, columns, M, aoii_penalty):
    
    last_update_times = {col: 0 for col in columns}  # Last update time for each node
    state_node = {col: np.array([20.0, 0.1]) for col in columns}  # Node states

    for t in range(len(pivot_df)):
        # Step 1: Compute Whittle indices for each node based on AoII
        whittle_indices = {}
        for col in columns:
            last_state_value, last_rate_of_change = state_node[col]
            measured_value = pivot_df.loc[t, col]

            # Correct AoII calculation at the sink using rate of change
            current_aoii = calculate_aoii_sink(t, last_update_times[col], last_rate_of_change)
            future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], last_rate_of_change)
            future_aoii_active = 0  # AoII resets to 0 if polled

            # Whittle index calculations
            q_passive = current_aoii + future_aoii_passive
            q_active = current_aoii + future_aoii_active + aoii_penalty
            whittle_indices[col] = q_passive - q_active

        # Step 2: Select top M nodes to poll based on Whittle indices
        nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
        if len(nodes_to_poll) > M:
            nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

        # Step 3: Poll selected nodes and calculate rewards
        for col in nodes_to_poll:
            measured_value = pivot_df.loc[t, col]
            last_state_value, last_rate_of_change = state_node[col]

            delta_t_dynamic = t - last_update_times[col]  # Time since last update

            # Update node state and last update time
            state_node[col] = update_node_state_dewma(
                measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1=beta_1, beta_2=beta_2
            )
            last_update_times[col] = t

            # Extract node ID dynamically and categorize
            node_id = extract_node_id(col)
            if node_id is not None:
                if 1 <= node_id <= 5:
                    category_counts['Category A'] += 1
                elif 6 <= node_id <= 10:
                    category_counts['Category B'] += 1


    return  category_counts

# Run the simulation
category_pulled_counts = run_simulation_whittle_aoii(
    pivot_df, columns, M, aoii_penalty
)

# Print the number of times each category was pulled
print("Transmission Count by Category:")
for category, count in category_pulled_counts.items():
    print(f"{category}: {count} times")




Transmission Count by Category:
Category A: 98 times
Category B: 1085 times


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load and restrict data
pivot_df = pd.read_csv("synthetic_temp_polling_data.csv").head(20000)

# Parameters
M = 5  # Max nodes to poll simultaneously
aoii_penalty = 0.5
beta_1 = 0.95
beta_2 = 0.1

columns = [col for col in pivot_df.columns if col != "SN"]

# Initialize tracking variables
last_update_times = {col: 0 for col in columns}
state_node = {col: np.array([20.0, 0.1]) for col in columns}
temp_min_max_history = {col: {'min': float('inf'), 'max': float('-inf')} for col in columns}
category_counts = {'Category A': 0, 'Category B': 0}

# Functions
def calculate_aoii_sink(current_time, last_received_time, normalised_rate_of_change):
    return abs((current_time - last_received_time) * normalised_rate_of_change)

def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t, beta_1, beta_2):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

def extract_node_id(col_name):
    digits = ''.join(filter(str.isdigit, col_name))
    return int(digits) if digits else None

# Main simulation loop
for t in range(len(pivot_df)):
    whittle_indices = {}

    for col in columns:
        measured_value = pivot_df.loc[t, col]

        # Update min/max dynamically for normalisation
        temp_min_max_history[col]['min'] = min(temp_min_max_history[col]['min'], measured_value)
        temp_min_max_history[col]['max'] = max(temp_min_max_history[col]['max'], measured_value)
        min_temp = temp_min_max_history[col]['min']
        max_temp = temp_min_max_history[col]['max']

        epsilon = 1e-6  # avoid division by zero
        normalised_temp = (measured_value - min_temp) / (max_temp - min_temp + epsilon)

        # Calculate normalised rate of change
        if t > 0:
            prev_value = pivot_df.loc[t-1, col]
            prev_norm_temp = (prev_value - min_temp) / (max_temp - min_temp + epsilon)
            normalised_rate_of_change = abs(normalised_temp - prev_norm_temp)
        else:
            normalised_rate_of_change = 0

        current_aoii = calculate_aoii_sink(t, last_update_times[col], normalised_rate_of_change)
        future_aoii_passive = calculate_aoii_sink(t+1, last_update_times[col], normalised_rate_of_change)
        future_aoii_active = 0

        q_passive = current_aoii + future_aoii_passive
        q_active = current_aoii + future_aoii_active + aoii_penalty
        whittle_indices[col] = q_passive - q_active

    # Poll decision
    nodes_to_poll = [col for col in whittle_indices if whittle_indices[col] >= 0]
    if len(nodes_to_poll) > M:
        nodes_to_poll = sorted(nodes_to_poll, key=whittle_indices.get, reverse=True)[:M]

    for col in nodes_to_poll:
        measured_value = pivot_df.loc[t, col]
        last_state_value, last_rate_of_change = state_node[col]

        delta_t_dynamic = t - last_update_times[col]

        state_node[col] = update_node_state_dewma(
            measured_value, last_state_value, last_rate_of_change, delta_t_dynamic, beta_1, beta_2
        )
        last_update_times[col] = t

        # Track polling count by category
        node_id = extract_node_id(col)
        if node_id:
            if 1 <= node_id <= 5:
                category_counts['Category A'] += 1
            elif 6 <= node_id <= 10:
                category_counts['Category B'] += 1

# Display polling counts
print("Transmission Count by Category:")
for category, count in category_counts.items():
    print(f"{category}: {count} times")

Transmission Count by Category:
Category A: 2192 times
Category B: 1340 times
